### チュートリアルの学習
- https://yutaroogawa.github.io/pytorch_tutorials_jp/

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [8]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # in_channels, out_channels, kernel_size,
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # 重みとバイアス in_features、 out_features、
        self.fc1 = nn.Linear(16 * 6 * 6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        # reshape flatにする
        x = x.view(-1, self.num_flat_features(x))

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [9]:
net = Net()
print(net)

params = list(net.parameters())
print(params)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
[Parameter containing:
tensor([[[[ 0.1408, -0.0704,  0.0047],
          [ 0.2208, -0.0701,  0.0765],
          [ 0.2978, -0.3073, -0.3067]]],


        [[[ 0.0817,  0.1571,  0.0691],
          [ 0.1167, -0.0291,  0.0223],
          [-0.0007, -0.2216,  0.1159]]],


        [[[ 0.3209, -0.1915, -0.1789],
          [-0.0391, -0.0098, -0.3120],
          [-0.3322, -0.2748, -0.1092]]],


        [[[-0.0595, -0.0412, -0.2800],
          [ 0.0006,  0.0248,  0.2554],
          [-0.3092, -0.2750,  0.2347]]],


        [[[ 0.0184,  0.2932,  0.1923],
          [ 0.0286, -0.2462,  0.0137],
          [-0.1259,  0.1049, -0.0844]]],


        [[[-0.1092,  0.0664,  0.2913],
          [-0.1673, -0.0

In [10]:
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 3, 3])


In [16]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(input)

tensor([[[[ 0.0270, -1.3795, -0.3806,  ..., -0.9540,  1.7448,  0.4794],
          [-0.8148, -0.3845,  0.0347,  ...,  1.2846,  1.7348, -0.7447],
          [ 0.0965,  0.8294,  0.1964,  ..., -1.3747, -0.0657, -1.4138],
          ...,
          [-1.5689, -1.7005,  0.8492,  ..., -0.4225, -0.7189,  0.4458],
          [-0.9486, -1.6094,  0.3247,  ...,  0.4962,  0.8666,  1.1247],
          [ 0.3666,  1.9676,  1.0934,  ..., -0.1110,  1.4365, -1.0606]]]])


In [17]:
net.zero_grad()
out.backward(torch.randn(1, 10))

In [18]:
output = net(input)
target = torch.randn(10)
target = target.view(1, -1)
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(1.2431, grad_fn=<MseLossBackward0>)


In [19]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [15]:
net.zero_grad()     # netのパラメータの勾配をゼロにする

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0109, -0.0086,  0.0041, -0.0113, -0.0051, -0.0065])


In [20]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.01)

# 訓練ループの中で以下を実行:
optimizer.zero_grad()
output = net(input)
# criterion = nn.MSELoss()
loss = criterion(output, target)
loss.backward()
optimizer.step()

